<a href="https://colab.research.google.com/github/matteoalberti/Lectures_AdvCV_Experis2020/blob/main/day2/DL4CV3_Though_pretrained_models_vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://github.com/matteoalberti/Lectures_introCV_Experis2020/blob/main/images/intro.jpg?raw=1)

# **Welcome!**

# Introduction to Machine Learning for Computer Vision

#### Thought Pre-Trained Models!


## **Lecturer :** Matteo Alberti


*Contacts :* https://www.linkedin.com/in/matteo-alberti-170493/


# Summary


- <font color=E35F2A>**Transfer Learning & Pre-Trained Models** </font>


#### Load Packages

In [80]:
#Sklearn ML
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import numpy as np
import platform
import keras

#TF Imports
import tensorflow as tf
import sklearn.model_selection as model_selection
import datetime
import pandas as pd


#Visualization
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

print('TF Version : ',tf.__version__)
print('Python Version : ', platform.python_version())

TF Version :  2.3.0
Python Version :  3.6.9


## Load Cifar100 this time!

This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).

Divided in **Supercalsses and Classes**

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

#The range for each individual colour is 0-255
x_train = x_train.astype('float32')/255 
x_test = x_test.astype('float32')/255

169009152/169001437 [==============================] - 2s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [ ]:
np.unique(y_train)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [ ]:
# check size
x_train.size // 1024 // 1024

146

In [ ]:
len(x_train)

50000

##### Visulize Data & Stats

### Define HyperParameters & CNN Architecture
   

In [4]:
#Parameters
import sys;import argparse; sys.argv=['']; del sys
parser = argparse.ArgumentParser(description="CNN")
parser.add_argument('--epochs', default=20, type=int)
parser.add_argument('--iter', default=100, type=int)
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--lr', default=0.001, type=float)

#For VGG
parser.add_argument('--weight_decay', default=0.0001, type=float)
parser.add_argument('--dropout', default=0.5, type=float)


parser.add_argument('--height', default=32, type=int)
parser.add_argument('--width', default=32, type=int)
parser.add_argument('--channel', default=3, type=int)
parser.add_argument('--classes', default=10, type=int)

#FOR RESNET
parser.add_argument('--stack_n', type=int, default=25, metavar='NUMBER',
                help='stack number n, total layers = 6 * n + 2 (default: 5)')

parser.add_argument('--train', default=False)
args = parser.parse_args()

layers_res = 6 * args.stack_n + 2
parser.add_argument('--layers_res', default=layers_res)

#Extras
parser.add_argument('--early_stop', default=3)


args = parser.parse_args()

print(args)

Namespace(batch_size=128, channel=3, classes=10, dropout=0.5, early_stop=3, epochs=20, height=32, iter=100, layers_res=152, lr=0.001, stack_n=25, train=False, weight_decay=0.0001, width=32)


# Thought deep pre-trained models - [Part 1]

### Visual Geometry Group Network 

<img src="https://www.researchgate.net/profile/Max_Ferguson/publication/322512435/figure/fig3/AS:697390994567179@1543282378794/Fig-A1-The-standard-VGG-16-network-architecture-as-proposed-in-32-Note-that-only.png" alt="Drawing" style="width: 500px;"/>

### Start with loading weights

- Which weights can I load?
- What have I to modify?

In [16]:
vgg16 = tf.keras.applications.VGG16(
    include_top=False, weights='imagenet', input_tensor=None, input_shape=(32,32,3),
    pooling=None, classes=100, classifier_activation='softmax')


vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

### Evaluate how good is our model without training!


In [7]:
score = vgg16.evaluate(x_test, y_test, verbose=0)
print(score[0], ' : loss')
print(score[1]*100, '% : acc')

# Why?? What happened???

16.370681762695312  : loss
0.0 % : acc


## Let's go step-by-step

### Define layers to be added

In [9]:
flatten = tf.keras.layers.Flatten()
dense1 = tf.keras.layers.Dense(200, activation='relu')
dense2 = tf.keras.layers.Dense(200, activation='relu')
classificator = tf.keras.layers.Dense(100, activation='softmax')

### Re-define our model

strato1 = operazione1(input)
strato2 = operazione2(strato1)
. . . 


In [19]:
inputs = tf.keras.Input(shape=(32,32, 3))
x1 = vgg16(inputs)
x2 = tf.keras.layers.Flatten()(x1)
x3 = dense1(x2)
x4 = dense2(x3)
outputs = classificator(x4)

vgg16_F = tf.keras.Model(inputs, outputs)

In [12]:
vgg16_F.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 200)               102600    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
Total params: 14,877,588
Trainable params: 14,877,588
Non-trainable params: 0
__________________________________________

#### RE-compile and summary

In [14]:
#Compile architecture
vgg16_F.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [15]:
# define callbacks

%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# start train 
history_VGG_provafake = vgg16_F.fit(x=x_train, 
          y=y_train, 
          epochs=5,
          validation_split=0.2, 
          callbacks=[tensorboard_callback], verbose=1)

score = vgg16_F.evaluate(x_test, y_test, verbose=0)
print(score[0], ' : loss')
print(score[1]*100, '% : acc')

# And now? What should we do?

![](https://drive.google.com/uc?export=view&id=1k9_HZH4LhYpbseIvRgQ-1ko6UF4C_hEb)


### Freeze Parameters

In [20]:
vgg16.trainable = False
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [24]:
vgg16_F.trainable = True
vgg16.trainable = False

vgg16_F.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 200)               102600    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
Total params: 14,877,588
Trainable params: 162,900
Non-trainable params: 14,714,688
____________________________________

## Evaluate directly!

In [ ]:
# define callbacks

%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# start train 
history_VGG16_4 = vgg16_F.fit(x=x_train, 
          y=y_train, 
          epochs=5,
          validation_split=0.2, 
          callbacks=[tensorboard_callback], verbose=1)

score = vgg16_F.evaluate(x_test, y_test, verbose=0)
print(score[0], ' : loss')
print(score[1]*100, '% : acc')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/5
1250/1250 [==============================] - 18s 14ms/step - loss: 2765401.5000 - accuracy: 0.0106 - val_loss: 4.6050 - val_accuracy: 0.0104
Epoch 2/5
1250/1250 [==============================] - 18s 14ms/step - loss: 4.6055 - accuracy: 0.0098 - val_loss: 4.6056 - val_accuracy: 0.0086
Epoch 3/5
1250/1250 [==============================] - 18s 14ms/step - loss: 4.6055 - accuracy: 0.0101 - val_loss: 4.6064 - val_accuracy: 0.0084
Epoch 4/5
1250/1250 [==============================] - 18s 14ms/step - loss: 4.6056 - accuracy: 0.0098 - val_loss: 4.6071 - val_accuracy: 0.0084
Epoch 5/5
1250/1250 [==============================] - 18s 14ms/step - loss: 4.6056 - accuracy: 0.0095 - val_loss: 4.6074 - val_accuracy: 0.0084
4.6053972244262695  : loss
0.9999999776482582 % : acc


## Transfer Learning as Weights Initialization

In [25]:
vgg16_WI = vgg16_F
vgg16_WI.trainable = True
vgg16_WI.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 200)               102600    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
Total params: 14,877,588
Trainable params: 14,877,588
Non-trainable params: 0
__________________________________________

In [ ]:
# define callbacks

%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# start train 
history_VGG = vgg19_WI.fit(x=x_train, 
          y=y_train, 
          epochs=5,
          validation_split=0.2, 
          callbacks=[tensorboard_callback], verbose=1)

score = vgg19_WI.evaluate(x_test, y_test, verbose=0)
print(score[0], ' : loss')
print(score[1]*100, '% : acc')

![](https://github.com/matteoalberti/Lectures_introCV_Experis2020/blob/main/images/DL4CV1.png?raw=1)

## Transfer Learning as Features Extractor

In [28]:
pd.set_option('max_colwidth', -1)

layers = [(layer.name, layer.trainable) for layer in vgg16_WI.layers]

layers_initial = [(layer.name, layer.trainable) for layer in vgg16.layers]

vgg_initial = pd.DataFrame(layers_initial, columns=['Layer Name', 'Layer Trainable'])
vgg_model = pd.DataFrame(layers, columns=['Layer Name', 'Layer Trainable'])

vgg_trainable = pd.concat([vgg_initial, vgg_model[2:]])

vgg_trainable.head(100)

__main__:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,Layer Name,Layer Trainable
0,input_6,True
1,block1_conv1,True
2,block1_conv2,True
3,block1_pool,True
4,block2_conv1,True
5,block2_conv2,True
6,block2_pool,True
7,block3_conv1,True
8,block3_conv2,True
9,block3_conv3,True


## Freeze only some layers

In [30]:
vgg16.layers[:11]

In [31]:
# Freeze all the layers before the `fine_tune_at` layer
for layer in vgg16.layers[:11]:
  layer.trainable =  False


pd.set_option('max_colwidth', -1)

layers = [(layer.name, layer.trainable) for layer in vgg16_WI.layers]
layers_initial = [(layer.name, layer.trainable) for layer in vgg16.layers]

vgg_initial = pd.DataFrame(layers_initial, columns=['Layer Name', 'Layer Trainable'])
vgg_model = pd.DataFrame(layers, columns=['Layer Name', 'Layer Trainable'])

vgg_trainable = pd.concat([vgg_initial, vgg_model[2:]])

vgg_trainable.head(100)

__main__:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,Layer Name,Layer Trainable
0,input_6,False
1,block1_conv1,False
2,block1_conv2,False
3,block1_pool,False
4,block2_conv1,False
5,block2_conv2,False
6,block2_pool,False
7,block3_conv1,False
8,block3_conv2,False
9,block3_conv3,False


In [32]:
vgg16_FE = vgg16_WI

vgg16_FE.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 200)               102600    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
Total params: 14,877,588
Trainable params: 13,142,100
Non-trainable params: 1,735,488
__________________________________

## Exercise

In [ ]:
# loading cifar100

# define VGG11 and train

# save_weights (con model.save_weights)



## Applichiamo su un nuovo dataset. 

      cifar10

*domanda*

cifar10  simile o no a cifra100? cosa dobbiamo cambiare??

In [4]:
(x_train10, y_train10), (x_test10, y_test10) = tf.keras.datasets.cifar10.load_data()

print(x_train10.shape, y_train10.shape, x_test10.shape, y_test10.shape)

#The range for each individual colour is 0-255
x_train10 = x_train10.astype('float32')/255 
x_test10 = x_test10.astype('float32')/255

170500096/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [5]:
(x_train100, y_train100), (x_test100, y_test100) = tf.keras.datasets.cifar100.load_data()

print(x_train100.shape, y_train100.shape, x_test100.shape, y_test100.shape)

#The range for each individual colour is 0-255
x_train100 = x_train100.astype('float32')/255 
x_test100 = x_test100.astype('float32')/255

169009152/169001437 [==============================] - 5s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [6]:
# redefine modello2 (num_classes)

# load_weights (modello2.load_weights(percorso))

# compile


# Choose! 1-2-3-4?

# model.fit

# model.evaluate

In [7]:
#Instantiate an empty model

def VGG_blocks1(n_conv, rec_field, act):
  return tf.keras.models.Sequential([    
  tf.keras.layers.Conv2D(n_conv, rec_field, padding='same', activation=act),
  tf.keras.layers.Conv2D(n_conv, rec_field, activation=act, padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))])

def VGG_blocks2(n_conv, rec_field, act):
  return tf.keras.models.Sequential([    
  tf.keras.layers.Conv2D(n_conv, rec_field, padding='same', activation=act),
  tf.keras.layers.Conv2D(n_conv, rec_field, padding='same', activation=act),
  tf.keras.layers.Conv2D(n_conv, rec_field, padding='same', activation=act),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))])

def CLASSIFICATION(n_dense, n_classes):
  return tf.keras.models.Sequential([
  tf.keras.layers.Dense(n_dense, activation='relu'),
  tf.keras.layers.Dense(n_dense, activation='relu'),
  tf.keras.layers.Dense(n_classes, activation='softmax')])

In [8]:
inputs = tf.keras.Input(shape=(32,32,3))

#features extractor
x = VGG_blocks1(n_conv=64, rec_field=(3,3), act='relu')(inputs)
x = VGG_blocks1(n_conv=128, rec_field=(3,3), act='relu')(x)
x = VGG_blocks2(n_conv=256, rec_field=(3,3), act='relu')(x)
x = VGG_blocks2(n_conv=512, rec_field=(3,3), act='relu')(x)

#classification
x = tf.keras.layers.Flatten()(x)
outputs = CLASSIFICATION(n_dense=256, n_classes=100)(x)


VGG_EASY = tf.keras.Model(inputs, outputs)


#Compile architecture
VGG_EASY.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

VGG_EASY.layers[1]._name = "first_conv_block"
VGG_EASY.layers[2]._name = "second_conv_block_2"
VGG_EASY.layers[3]._name = "3rd_conv_block_3"
VGG_EASY.layers[4]._name = "4th_conv_block_3"
VGG_EASY.layers[6]._name = "classification_block_3"

VGG_EASY.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
first_conv_block (Sequential (None, 16, 16, 64)        38720     
_________________________________________________________________
second_conv_block_2 (Sequent (None, 8, 8, 128)         221440    
_________________________________________________________________
3rd_conv_block_3 (Sequential (None, 4, 4, 256)         1475328   
_________________________________________________________________
4th_conv_block_3 (Sequential (None, 2, 2, 512)         5899776   
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
classification_block_3 (Sequ (None, 100)              

In [80]:
# define callbacks

%reload_ext tensorboard
log_dir = "logs/fit/base_cifar100" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# start train 
history_VGG_cifar100 = VGG_EASY.fit(x=x_train100, 
          y=y_train100, 
          epochs=30,
          validation_split=0.2, 
          callbacks=[tensorboard_callback], verbose=1)

score = VGG_EASY.evaluate(x_test100, y_test100, verbose=0)
print(score[0], ' : loss on cifar100')
print(score[1]*100, '% : acc on cifar100')

In [ ]:
! mkdir weights

In [ ]:
# export weights

VGG_EASY.save_weights('weights/VGG11_cifar100.h5')

### Which case???

![](https://drive.google.com/uc?export=view&id=1k9_HZH4LhYpbseIvRgQ-1ko6UF4C_hEb)



## Let's try all!

*Test on cifar10!*

In [86]:
inputs = tf.keras.Input(shape=(32,32,3))

#features extractor
x = VGG_blocks1(n_conv=64, rec_field=(3,3), act='relu')(inputs)
x = VGG_blocks1(n_conv=128, rec_field=(3,3), act='relu')(x)
x = VGG_blocks2(n_conv=256, rec_field=(3,3), act='relu')(x)
x = VGG_blocks2(n_conv=512, rec_field=(3,3), act='relu')(x)

#classification
x = tf.keras.layers.Flatten()(x)
outputs = CLASSIFICATION(n_dense=256, n_classes=100)(x)


VGG_WI = tf.keras.Model(inputs, outputs)


#Compile architecture
VGG_WI.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

VGG_WI.layers[1]._name = "first_conv_block"
VGG_WI.layers[2]._name = "second_conv_block_2"
VGG_WI.layers[3]._name = "3rd_conv_block_3"
VGG_WI.layers[4]._name = "4th_conv_block_3"
VGG_WI.layers[6]._name = "classification_block_3"

VGG_WI.summary()

Model: "functional_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
first_conv_block (Sequential (None, 16, 16, 64)        38720     
_________________________________________________________________
second_conv_block_2 (Sequent (None, 8, 8, 128)         221440    
_________________________________________________________________
3rd_conv_block_3 (Sequential (None, 4, 4, 256)         1475328   
_________________________________________________________________
4th_conv_block_3 (Sequential (None, 2, 2, 512)         5899776   
_________________________________________________________________
flatten_14 (Flatten)         (None, 2048)              0         
_________________________________________________________________
classification_block_3 (Sequ (None, 100)             

In [87]:
# load weights
VGG_WI.load_weights('weights/VGG11_cifar100.h5')

In [88]:
VGG_WI_notop = tf.keras.Sequential(VGG_WI.layers[:-1])
VGG_WI_notop.summary()

Model: "sequential_61"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_conv_block (Sequential (None, 16, 16, 64)        38720     
_________________________________________________________________
second_conv_block_2 (Sequent (None, 8, 8, 128)         221440    
_________________________________________________________________
3rd_conv_block_3 (Sequential (None, 4, 4, 256)         1475328   
_________________________________________________________________
4th_conv_block_3 (Sequential (None, 2, 2, 512)         5899776   
_________________________________________________________________
flatten_14 (Flatten)         (None, 2048)              0         
Total params: 7,635,264
Trainable params: 7,635,264
Non-trainable params: 0
_________________________________________________________________


In [90]:
inputs = tf.keras.Input(shape=(32,32,3))

#features extractor
x = VGG_WI_notop(inputs)

#classification
outputs = CLASSIFICATION(n_dense=10, n_classes=10)(x)


VGG_WI_dense = tf.keras.Model(inputs, outputs)


#Compile architecture
VGG_WI_dense.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

VGG_WI_dense.layers[1]._name = "VGG11"
VGG_WI_dense.layers[2]._name = "Classification"

VGG_WI_dense.summary()

Model: "functional_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
VGG11 (Sequential)           (None, 2048)              7635264   
_________________________________________________________________
Classification (Sequential)  (None, 10)                20710     
Total params: 7,655,974
Trainable params: 7,655,974
Non-trainable params: 0
_________________________________________________________________


### We are ready for training

**First scenario**


      train everything

In [91]:
VGG_WI_dense.trainable = True

In [92]:
VGG_WI_dense.summary()

Model: "functional_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
VGG11 (Sequential)           (None, 2048)              7635264   
_________________________________________________________________
Classification (Sequential)  (None, 10)                20710     
Total params: 7,655,974
Trainable params: 7,655,974
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# define callbacks

%reload_ext tensorboard
log_dir = "logs/fit/cifar10_base_1" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# start train 
history_VGG_cifar10_1 = VGG_WI_dense.fit(x=x_train10, 
          y=y_train10, 
          epochs=30,
          validation_split=0.2, 
          callbacks=[tensorboard_callback], verbose=1)


In [ ]:
score_1 = VGG_WI_dense.evaluate(x_test10, y_test10, verbose=0)
print(score_1[0], ' : loss on cifar100')
print(score_1[1]*100, '% : acc on cifar100')

**4th case**

In [93]:
VGG_WI.trainable = False
VGG_WI_dense.summary()

Model: "functional_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
VGG11 (Sequential)           (None, 2048)              7635264   
_________________________________________________________________
Classification (Sequential)  (None, 10)                20710     
Total params: 7,655,974
Trainable params: 20,710
Non-trainable params: 7,635,264
_________________________________________________________________


In [ ]:
# define callbacks

%reload_ext tensorboard
log_dir = "logs/fit/cifar10_base_4" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# start train 
history_VGG_cifar10_4 = VGG_WI_dense.fit(x=x_train10, 
          y=y_train10, 
          epochs=30,
          validation_split=0.2, 
          callbacks=[tensorboard_callback], verbose=1)


In [ ]:
score_4 = VGG_WI_dense.evaluate(x_test10, y_test10, verbose=0)
print(score_1[0], ' : loss on cifar100')
print(score_1[1]*100, '% : acc on cifar100')

**Second and Third one**

In [106]:
# re-initialize
VGG_WI_dense.trainable = True
VGG_WI_notop.trainable = True
VGG_WI.trainable = True

In [103]:
VGG_WI_notop.summary()

Model: "VGG11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_conv_block (Sequential (None, 16, 16, 64)        38720     
_________________________________________________________________
second_conv_block_2 (Sequent (None, 8, 8, 128)         221440    
_________________________________________________________________
3rd_conv_block_3 (Sequential (None, 4, 4, 256)         1475328   
_________________________________________________________________
4th_conv_block_3 (Sequential (None, 2, 2, 512)         5899776   
_________________________________________________________________
flatten_14 (Flatten)         (None, 2048)              0         
Total params: 7,635,264
Trainable params: 7,635,264
Non-trainable params: 0
_________________________________________________________________


In [107]:
# Freeze all the layers before the `fine_tune_at` layer
for layer in VGG_WI_notop.layers[:2]:
  layer.trainable =  False


pd.set_option('max_colwidth', -1)

layers = [(layer.name, layer.trainable) for layer in VGG_WI_dense.layers]
layers_initial = [(layer.name, layer.trainable) for layer in VGG_WI.layers]

vgg_initial = pd.DataFrame(layers_initial, columns=['Layer Name', 'Layer Trainable'])
vgg_model = pd.DataFrame(layers, columns=['Layer Name', 'Layer Trainable'])

vgg_trainable = pd.concat([vgg_initial, vgg_model[2:]])

vgg_trainable.head(100)

__main__:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,Layer Name,Layer Trainable
0,input_23,True
1,first_conv_block,False
2,second_conv_block_2,False
3,3rd_conv_block_3,True
4,4th_conv_block_3,True
5,flatten_14,True
6,classification_block_3,True
2,Classification,True


In [108]:
VGG_WI_dense.summary()

Model: "functional_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
VGG11 (Sequential)           (None, 2048)              7635264   
_________________________________________________________________
Classification (Sequential)  (None, 10)                20710     
Total params: 7,655,974
Trainable params: 7,395,814
Non-trainable params: 260,160
_________________________________________________________________


In [ ]:
# define callbacks

%reload_ext tensorboard
log_dir = "logs/fit/cifar10_base_3" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# start train 
history_VGG_cifar10_2_3 = VGG_WI_dense.fit(x=x_train10, 
          y=y_train10, 
          epochs=30,
          validation_split=0.2, 
          callbacks=[tensorboard_callback], verbose=1)


In [ ]:
score_2_3 = VGG_WI_dense.evaluate(x_test10, y_test10, verbose=0)
print(score_1[0], ' : loss on cifar100')
print(score_1[1]*100, '% : acc on cifar100')

## We need to understand the main differences


    1 v 2
    1 vs 3
    1 vs 4

    2 vs 3
    2 vs 4

    3 vs 4


*Ma se noi avessimo addestrato cifar10 su vgg11 da zero? che performance?*

In [ ]:
score_1 = VGG_WI_dense.evaluate(x_test10, y_test10, verbose=0)
print('\nFirst case')
print(score_1[0], ' : loss on cifar100')
print(score_1[1]*100, '% : acc on cifar100')

print('\nSecond_third case')
score_2_3 = VGG_WI_dense.evaluate(x_test10, y_test10, verbose=0)
print(score_1[0], ' : loss on cifar100')
print(score_1[1]*100, '% : acc on cifar100')

print('\nLast case')
score_4 = VGG_WI_dense.evaluate(x_test10, y_test10, verbose=0)
print(score_1[0], ' : loss on cifar100')
print(score_1[1]*100, '% : acc on cifar100')

## Channels? 

domanda :

      - ho un modello pre-trainato su immagini BN. voglio applicarlo ad un dataset con immagini a colori

      - ho un modello pre-trainato su immagini RGB (colori). Voglio applicarlo su un dataset BN

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

#The range for each individual colour is 0-255
x_train = x_train.astype('float32')/255 
x_test = x_test.astype('float32')/255